VEP Analysis Code
AJK
02-16-2024

In [74]:
import mne
import numpy as np
from mne.preprocessing import (ICA)
from autoreject import AutoReject
import matplotlib
matplotlib.use("TkAgg")

eeg_path = "David_VEP"  # You will need to change this location

file_eeg = eeg_path + ".eeg"
file_vhdr = eeg_path + ".vhdr"
file_vmrk = eeg_path + ".vmrk"

raw = mne.io.read_raw_brainvision(file_vhdr)
drop_channels = ['BIP2','EOG','TEMP1','ACC1','ACC2','ACC3']
raw = raw.drop_channels(drop_channels)
events_from_annot, event_dict = mne.events_from_annotations(raw)
del event_dict['Stimulus/s5']
raw.set_channel_types({'BIP1':'ecg'})
raw.info
raw.plot()


Extracting parameters from David_VEP.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['Marker/Impedance', 'New Segment/', 'Stimulus/s1', 'Stimulus/s2', 'Stimulus/s3', 'Stimulus/s5']
Opening raw-browser...


/var/folders/dv/s4k5_3r51ml3q36bgbcdsgbr0000gn/T/ipykernel_35896/738744482.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_vhdr)


<MNEBrowseFigure size 1296x1017.33 with 4 Axes>

In [75]:
highpass = 0.5
lowpass = 50
notch = 60

raw_filtered = raw.load_data().filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))
#raw_filtered = raw.resample(resample).filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))
eeg_1020 = raw_filtered.copy().set_eeg_reference(ref_channels = 'average') #['Fz'])
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
eeg_1020 = eeg_1020.set_montage(ten_twenty_montage, on_missing = 'ignore')
del raw, raw_filtered, ten_twenty_montage
eeg_1020.info['bads'] = []
picks = mne.pick_types(eeg_1020.info, meg=False, eeg=True, stim=False, eog=False, include=[], exclude=[])

epochs = mne.Epochs(eeg_1020,
                    events=events_from_annot,
                    event_id=event_dict,
                    tmin=-0.1,
                    tmax=0.3,   #duration of stimulus or response
                    baseline=None,
                    reject=None,
                    verbose=False,
                    preload=True,
                    detrend=None,
                    event_repeated='drop')


Reading 0 ... 95205  =      0.000 ...   190.410 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 3301 samples (6.602 sec)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3301 samples (6.602 sec)

EE

In [76]:
n_interpolates = np.array([1, 4, 32])
consensus_percs = np.linspace(0, 1.0, 11)
ar = AutoReject(n_interpolates,
                consensus_percs,
                picks=picks,
                thresh_method='random_search',
                random_state=42)    #random n state
epochs_ar = ar.fit_transform(epochs)

ica = ICA(n_components = 16, max_iter = 'auto', random_state = 123)
ica.fit(epochs_ar)
ica_z_thresh = 1.96

epochs_clean = epochs_ar.copy()
ecg_indices, ecg_scores = ica.find_bads_ecg(epochs_clean,
                                            threshold=ica_z_thresh)
ica.exclude = ecg_indices
ica.plot_scores(ecg_scores)
print(ecg_indices)
ica.apply(epochs_clean)

#ica.fit(epochs_clean2)
#eog_indices, eog_scores = ica.find_bads_eog(epochs_clean,
#                                            ch_name=['Fp1', 'F6'],
#                                            threshold=ica_z_thresh)
#ica.exclude = eog_indices
#print(eog_indices)
#ica.apply(epochs_clean)

epochs_final = epochs_clean.copy()
epochs_final.plot()
del eeg_1020, epochs, epochs_clean


Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/63 [00:00<?,       ?it/s]

100%|██████████| Creating augmented epochs : 63/63 [00:25<00:00,    2.51it/s]
100%|██████████| Computing thresholds ... : 63/63 [00:08<00:00,    7.83it/s]























100%|██████████| Repairing epochs : 159/159 [00:00<00:00,  410.67it/s]



















































































































































100%|██████████| Repairing epochs : 159/159 [00:33<00:00,    4.72it/s]






















100%|██████████| Fold : 10/10 [00:01<00:00,    5.97it/s]


































































































































100%|██████████| Repairing epochs : 159/159 [00:13<00:00,   11.82it/s]






















100%|██████████| Fold : 10/10 [00:01<00:00,    6.33it/s]








































































































































100%|██████████| Repairing epochs : 15





Estimated consensus=0.10 and n_interpolate=4



















































































































































100%|██████████| Repairing epochs : 159/159 [00:13<00:00,   11.79it/s]

Dropped 22 epochs: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 19, 49, 50, 52, 58, 59, 118
Fitting ICA to data using 63 channels (please be patient, this may take a while)


Selecting by number: 16 components
Fitting ICA took 29.8s.
[]
Applying ICA to Epochs instance
    Transforming to ICA space (16 components)
    Zeroing out 0 ICA components
    Projecting back using 63 PCA components
Opening epochs-browser...


In [ ]:
########################################
#### Manual ICA Analysis #####
##ica.plot_sources(epochs)
#ica.plot_components()
#ica.plot_properties(epochs)
#exclude = [0,1,2]    # select based on ICA abnl. OPTIONAL.
#epochs_clean = epochs.copy()
#ica.exclude = exclude
#ica.apply(epochs_clean)
#epochs_clean.plot(n_channels = len(epochs_clean))
#epochs_final = epochs_clean.copy()

## if no ICAs:
#epochs_final = epochs_clean.copy()
#del eeg_1020, epochs, epochs_clean

In [77]:
########################################
baseline_tmin, baseline_tmax = -0.1, 0
baseline = (baseline_tmin, baseline_tmax)

VEP = epochs_final['Stimulus/s1'].apply_baseline(baseline).average()
VEP_2 = epochs_final['Stimulus/s3'].apply_baseline(baseline).average()
blank = epochs_final['Stimulus/s2'].apply_baseline(baseline).average()

fig = mne.viz.plot_compare_evokeds(VEP_2, picks=['Oz','O1','O2'], combine="mean", show=True)
fig[0].savefig("VEP_Occipital")

fig = mne.viz.plot_compare_evokeds(VEP_2, picks=['Oz'], show=True)
fig[0].savefig("VEP_Oz")

fig = mne.viz.plot_compare_evokeds(dict(Checkboard=VEP, Checkboard2=VEP_2, Blank=blank), colors=dict(Checkboard="orange", Blank="black"), picks=['Oz', 'O1', 'O2'], combine="mean")
fig[0].savefig("Compare_Stimuli_Occipital")


Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
combining channels using "mean"
Closing epochs-browser...
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
More than 6 channels, truncating title ...
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
